In [4]:
from pathlib import Path
import sys

ROOT = Path().resolve().parents[1] / "code"
sys.path.append(str(ROOT))
from functions.valid_recall import valid_popularity_recall
from functions.read_behaviors import read_train_behaviors,read_dev_behaviors

import pickle
from collections import defaultdict
import polars as pl

In [5]:
with open('/home/ming/GraduateProject/code/recall/recall_results/news_content_sim.pkl','rb') as f:
    news_sim=pickle.load(f)

In [6]:
news_sim

{'N60025': ['N37524',
  'N24313',
  'N88946',
  'N31958',
  'N44587',
  'N18999',
  'N11932',
  'N52101',
  'N38386',
  'N64537',
  'N30722',
  'N66859',
  'N10822',
  'N121759',
  'N29059',
  'N64077',
  'N104498',
  'N17881',
  'N120331',
  'N103332',
  'N76964',
  'N81969',
  'N115485',
  'N88260',
  'N60267',
  'N59012',
  'N68923',
  'N119801',
  'N4068',
  'N82097',
  'N74222',
  'N69817',
  'N23394',
  'N123478',
  'N86608',
  'N32183',
  'N110994',
  'N5785',
  'N44034',
  'N36306',
  'N76095',
  'N61496',
  'N73618',
  'N98562',
  'N13259',
  'N93011',
  'N124498',
  'N90410',
  'N127717',
  'N116456'],
 'N129849': ['N41607',
  'N7294',
  'N56331',
  'N95821',
  'N71215',
  'N6799',
  'N64758',
  'N54595',
  'N41591',
  'N71184',
  'N101449',
  'N64632',
  'N10360',
  'N23150',
  'N84463',
  'N17704',
  'N62613',
  'N17689',
  'N6515',
  'N86682',
  'N90566',
  'N34364',
  'N19971',
  'N47196',
  'N12099',
  'N91465',
  'N17410',
  'N4334',
  'N125940',
  'N127736',
  'N6668',

In [7]:
def recall_content_sim(pred_path=None, topk=50):
    if pred_path:
        pred = pl.read_parquet(pred_path)
    else:
        pred = read_dev_behaviors()

    pred = pred.select(['user_id', 'history'])

    rows = []

    for row in pred.iter_rows(named=True):
        uid = row['user_id']
        history = row['history'] or []

        hist = history
        score = defaultdict(float)

        for idx, nid in enumerate(reversed(hist)):
            if nid not in news_sim:
                continue
            w = 1.0 / (idx + 1)

            for sim_nid in news_sim[nid]:
                if sim_nid in history:
                    continue
                score[sim_nid] += w

        if not score:
            rows.append((uid, [], []))
            continue

        # 排序 + 截断
        sorted_items = sorted(score.items(), key=lambda x: x[1], reverse=True)[:topk]
        rec_list = [nid for nid, _ in sorted_items]
        rec_score = [s for _, s in sorted_items]

        rows.append((uid, rec_list, rec_score))

    return pl.DataFrame(
        rows,
        schema=["user_id", "rec_list", "rec_score"]
    )

In [8]:
res=recall_content_sim()

/home/ming/miniconda3/envs/rapids-25.12/lib/python3.10/functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


In [9]:
res

user_id,news_id,score
str,str,f64
"""U134050""","""N88638""",1.0
"""U134050""","""N86494""",1.0
"""U134050""","""N97114""",1.0
"""U134050""","""N11086""",1.0
"""U134050""","""N59639""",1.0
…,…,…
"""U655194""","""N2520""",1.0
"""U655194""","""N114181""",1.0
"""U655194""","""N25702""",1.0
